# Rocket Landing - Performance Analysis

In [2]:
!pip install torch torchview

  Using cached torch-2.6.0-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
Using cached torch-2.6.0-cp312-none-macosx_11_0_arm64.whl (66.5 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached setuptools-78.1.0-py3-none-any.whl (1.3

In [4]:
!pip install matplotlib
!pip install numpy
!pip install opencv-python
!pip install gymnasium
!pip install stable-baselines3

  Using cached matplotlib-3.10.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp312-cp312-macosx_10_13_universal2.whl.metadata (102 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.1-cp312-cp312-macosx_11_0_arm64.whl (8.0 MB)
Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl (255 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.57.0-cp312-cp312-macosx_10_13_universal2.whl (2.8 MB)
Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl (65 kB)
Using cached pillow-11.1.0-cp312-cp312-macosx_11_0_arm64.whl (3.1 MB)
Using cached pyparsing-3.2.3

In [5]:
import numpy as np
import torch
from rocket import Rocket
from policy import ActorCritic
import matplotlib.pyplot as plt
import utils
import os
import glob

In [12]:
# task = 'landing' # 'hover' or 'landing'
task = 'hover'   # 'hover' or 'landing'

max_steps = 800
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
env = Rocket(task=task, max_steps=max_steps)
ckpt_folder = os.path.join('./', task + '_ckpt')
net = ActorCritic(input_dim=env.state_dims, output_dim=env.action_dims).to(device)

print(glob.glob(os.path.join(ckpt_folder, '*.pt')))

if len(glob.glob(os.path.join(ckpt_folder, '*.pt'))) > 0:
    # load the last ckpt
    checkpoint = torch.load(glob.glob(os.path.join(ckpt_folder, '*.pt'))[-1])
    net.load_state_dict(checkpoint['model_G_state_dict'])
    last_episode_id = checkpoint['episode_id']
    REWARDS = checkpoint['REWARDS']

('mapping', PositionalMapping())
('linear1', Linear(in_features=120, out_features=128, bias=True))
('linear2', Linear(in_features=128, out_features=128, bias=True))
('linear3', Linear(in_features=128, out_features=128, bias=True))
('linear4', Linear(in_features=128, out_features=9, bias=True))
('relu', LeakyReLU(negative_slope=0.2))
[Linear(in_features=120, out_features=128, bias=True), Linear(in_features=128, out_features=128, bias=True), Linear(in_features=128, out_features=128, bias=True), Linear(in_features=128, out_features=9, bias=True)]
('mapping', PositionalMapping())
('linear1', Linear(in_features=120, out_features=128, bias=True))
('linear2', Linear(in_features=128, out_features=128, bias=True))
('linear3', Linear(in_features=128, out_features=128, bias=True))
('linear4', Linear(in_features=128, out_features=1, bias=True))
('relu', LeakyReLU(negative_slope=0.2))
[Linear(in_features=120, out_features=128, bias=True), Linear(in_features=128, out_features=128, bias=True), Linear

In [10]:
plt.figure(figsize=(16,10))
plt.plot(REWARDS), plt.plot(utils.moving_avg(REWARDS, N=50))
plt.legend(['episode reward', 'moving avg'], loc=2)
plt.xlabel('m episode')
plt.ylabel('reward')
# plt.savefig(os.path.join(ckpt_folder, 'rewards_' + str(episode_id).zfill(8) + '.jpg'))
plt.show()

NameError: name 'REWARDS' is not defined

<Figure size 1600x1000 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
episode = last_episode_id
window0 = int(episode/120)
window1 = int(episode/30)
yhat0 = savgol_filter(REWARDS, window0, 3) # window size window0, polynomial order 3
yhat1 = savgol_filter(REWARDS, window1, 1)
Aw = np.lib.stride_tricks.sliding_window_view(REWARDS, window1)
Aμ = np.mean(Aw, axis=-1)
Aσ = np.std(Aw, axis=-1)
print(episode,len(REWARDS),window0,window1,yhat0.shape,yhat1.shape,Aw.shape,Aμ.shape,Aσ.shape)

plt.figure(figsize=(16,10))
plt.plot(yhat0[window1//2:-window1//2])
# plt.plot(yhat1[window1//2:-window1//2])
plt.plot(Aμ)

plt.fill_between(range(Aμ.shape[0]),Aμ-Aσ,Aμ+Aσ,alpha=.1)
plt.title(f'AVERAGED REWARDS. {task.capitalize()} Episode: {episode}')
plt.show()

# Visualize Actor-Critic Network
https://stackoverflow.com/questions/52468956/how-do-i-visualize-a-net-in-pytorch

In [ ]:
import torchvision
"""
torchview 0.2.6
pip install torchview
Latest version: Feb 14, 2023
Visualization of Pytorch Models
Torchview provides visualization of pytorch models in the form of visual graphs.
Visualization includes tensors, modules, torch.functions and info such as input/output shapes.
"""
from torchview import draw_graph

model_graph = draw_graph(net, input_size=(1,env.state_dims), expand_nested=True)
model_graph.visual_graph.save("actor_critic.gv")
model_graph.visual_graph.render("actor_critic", format="png")
model_graph.visual_graph